In [159]:
import datetime
import dotenv
import json
import os
import psycopg2
import psycopg2.extras
import requests

dotenv.load_dotenv()

NYT_API_KEY = os.getenv("NYT_API_KEY")
POSTGRES_DATABASE = os.getenv("POSTGRES_DATABASE")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_USER = os.getenv("POSTGRES_USER")

def get_response(url):
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        # todo: Throw exception on failed request.
        print("Request failed with status code:", response.status_code)


# Year is YYYY; month is MM if between 10 and 12, else M.
def get_archive(year, month):
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={NYT_API_KEY}"
    return get_response(url)["response"]["docs"]


# psql postgresql://user:pw@host:port/db
def save_archive(archive):
    conn = psycopg2.connect(
        database = POSTGRES_DATABASE,
        host = POSTGRES_HOST,
        password = POSTGRES_PASSWORD,
        port = POSTGRES_PORT,
        user = POSTGRES_USER)

    with conn:
        with conn.cursor() as cursor:
            insert_query = "INSERT INTO archives_raw (article_json, article_jsonb) VALUES (%s, %s)"
            for article in archive:
                json_data = json.dumps(article)
                cursor.execute(insert_query, (json_data, json_data))

            conn.commit()

def get_connection():
    conn = psycopg2.connect(
        database = POSTGRES_DATABASE,
        host = POSTGRES_HOST,
        password = POSTGRES_PASSWORD,
        port = POSTGRES_PORT,
        user = POSTGRES_USER)
    cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

    return [conn, cursor]


def get_batch(size=5):
    conn, cursor = get_connection()
    with conn:
        with cursor:
            query = """
                SELECT 
                    REPLACE(article_json ->> 'uri', 'nyt://article/', '') AS id,
                    article_json -> 'headline' -> 'main' AS headline,
                    article_json -> 'abstract' AS abstract,
                    article_json -> 'lead_paragraph' AS lead_paragraph,
                    article_json -> 'byline' -> 'original' AS byline,
                    article_json -> 'type_of_material' AS type,
                    (article_json ->> 'pub_date')::timestamp AS pub_date,
                    article_json -> 'web_url' AS url
                FROM archives_raw
                WHERE (article_json ->> 'web_url') NOT IN (
                    SELECT url FROM articles
                );
            """
            cursor.execute(query)
            results = cursor.fetchmany(size)

    conn, cursor = get_connection()
    with conn:
        with cursor:
            insert_query = "INSERT INTO articles (id, headline, abstract, lead_paragraph, byline, type, pub_date, url) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
            for row in results:
                cursor.execute(insert_query, row)
            conn.commit()

    return results


In [160]:
def get_last_run():
    # Get completed run months from archives_raw table
        # Get oldest date (?min)
        # Subtract one month

    conn, cursor = get_connection()
    with conn:
        with cursor:
            query = """
                SELECT 
                    MIN((article_json ->> 'pub_date')::date) AS pub_date
                FROM archives_raw
            """
            cursor.execute(query)
            result = cursor.fetchone()
            
    return result[0]

def get_next_run():
    previous = get_last_run()
    if previous:
        next = previous - datetime.timedelta(weeks=4)
    else:
        next = datetime.datetime.strptime("2022-12-01", "%Y-%m-%d")

    return next

In [161]:
# next = get_next_run()
# print(next.year, next.month)

2022 12


In [162]:
# results = get_batch()

In [163]:
# archive = get_archive(2021, 1)
# save_archive(archive)

In [164]:
query_file = "sql/create_nyt_archive.sql"
with open(query_file, 'r') as file:
    query = file.read()

query

'CREATE TABLE IF NOT EXISTS nyt_archive (\n    article JSONB\n);'